In [ ]:
import numpy as np
import pandas as pd
import os
import time
import ast


# Read normal variables

In [ ]:
time0 = time.time()


def get_last_added_file(folder_path):
    # Get a list of all files in the folder
    files = os.listdir(folder_path)

    # Filter Excel files
    excel_files = [
        file
        for file in files
        if file.endswith(".xlsx") or file.endswith(".xls") or file.endswith(".csv")
    ]

    if not excel_files:
        return None  # No Excel files found
    print()
    # Sort files by modification time (newest first)
    excel_files.sort(
        key=lambda x: os.path.getctime(os.path.join(folder_path, x)), reverse=True
    )

    # Return the path to the last added Excel file
    return os.path.join(folder_path, excel_files[0])


def create_folder_if_not_exists(folder_name):
    # Get current folder
    folder_path = os.getcwd()

    try:
        os.makedirs("pip")
        print("DONE")
    except FileExistsError:
        pass
    if not os.path.exists(os.path.join(folder_path, folder_name)):
        os.makedirs(folder_name)
        print("Folder '{}' created.".format(folder_name))
    else:
        print("Folder '{}' already exists.".format(folder_name))


# Folder name to create
folder_name_to_convert = "File_To_Convert"
folder_name_converted = "File_Converted"

# Get current folder
folder_path = os.getcwd()


# Create new folder
create_folder_if_not_exists(folder_name_converted)


In [ ]:
# Get the last upload excel file
real_folder_path = os.path.join(folder_path, folder_name_to_convert)

last_file = get_last_added_file(real_folder_path)


if last_file:
    print("Last added file:", last_file)
else:
    print("No files found in the folder.")

# Get the correct naming for the converted file
filename_with_extension = os.path.basename(last_file)
splited_filename_with_extension = os.path.splitext(filename_with_extension)
print(splited_filename_with_extension)
filename = (
    splited_filename_with_extension[0]
    + "converted"
    + splited_filename_with_extension[1]
)

# Possibility to create a thread to run all the code during the wait for the user answer.
# scenario = input("Give the name of the scenario")
scenario = splited_filename_with_extension[0]

# Read the selected file
if (
    splited_filename_with_extension[1] == ".xls"
    or splited_filename_with_extension[1] == ".xlsx"
):
    scenario_variable_df = pd.read_excel(last_file)
elif splited_filename_with_extension[1] == ".csv":
    scenario_variable_df = pd.read_csv(last_file)


scenario_variable_df


In [ ]:
# Check for duplicate rows
duplicate_rows = scenario_variable_df[scenario_variable_df.duplicated()]

# Display duplicate rows
print("Duplicate Rows:")
print(duplicate_rows)

# Count the number of duplicate rows
num_duplicate_rows = duplicate_rows.shape[0]
print("Number of Duplicate Rows:", num_duplicate_rows)

scenario_variable_df = scenario_variable_df.drop_duplicates()
print(len(scenario_variable_df))

In [ ]:
# Rename the column of the file
columns = scenario_variable_df.columns.str.replace("Unnamed:", "Subscript")
columns = columns.str.replace(".", " ")

# Define the string you want in column names
search_string = "Subscript"
counter = 0

for k in range(len(columns)):
    if search_string in columns[k]:
        columns.values[k] = search_string + " " + str(counter)
        counter += 1


# Create a dictionary using zip() and dictionary comprehension
my_dict = {k: v for k, v in zip(scenario_variable_df.columns[2:6], columns[2:6])}
scenario_variable_df.rename(columns=my_dict, inplace=True)
scenario_variable_df.rename(
    columns={"Time": "Variable", "Year": "Unit", "Subscript": "Subscript 0"},
    inplace=True,
)

# Insert the three columns in the right place for IAMC format
scenario_variable_df.insert(0, "Model", "WILIAM")
scenario_variable_df.insert(1, "Scenario", scenario)
scenario_variable_df.insert(2, "Region", "World")


In [ ]:
# Dict used to check if the subscripts are a country
country_dict = {
    "EU27": 1,
    "UK": 1,
    "CHINA": 1,
    "EASOC": 1,
    "INDIA": 1,
    "LATAM": 1,
    "RUSSIA": 1,
    "USMCA": 1,
    "LROW": 1,
    "AUSTRIA": 1,
    "BELGIUM": 1,
    "BULGARIA": 1,
    "CROATIA": 1,
    "CYPRUS": 1,
    "CZECH_REPUBLIC": 1,
    "DENMARK": 1,
    "ESTONIA": 1,
    "FINLAND": 1,
    "FRANCE": 1,
    "GERMANY": 1,
    "GREECE": 1,
    "HUNGARY": 1,
    "IRELAND": 1,
    "ITALY": 1,
    "LATVIA": 1,
    "LITHUANIA": 1,
    "LUXEMBOURG": 1,
    "MALTA": 1,
    "NETHERLANDS": 1,
    "POLAND": 1,
    "PORTUGAL": 1,
    "ROMANIA": 1,
    "SLOVAKIA": 1,
    "SLOVENIA": 1,
    "SPAIN": 1,
    "SWEDEN": 1,
}


# Aggregate the subscripts at the end of the variable
def aggregate_variable_name(row, counter):
    for k in range(counter):
        subscript = row["Subscript " + str(k)]

        if k == 0:
            if country_dict.get(subscript) == 1:
                # Give the name of the region for that variable
                row["Region"] = subscript

                row["Subscript 0"] = np.nan
                continue

        if not pd.isnull(subscript):
            row["Variable"] = row["Variable"] + "|" + subscript

    return row


# Apply the previous function on all the lines
scenario_variable_df = scenario_variable_df.apply(
    aggregate_variable_name, args=(counter,), axis=1
)

# Remove the subscript columns once they have been added at the end of the variable name
drop_columns_list = ["Subscript " + str(k) for k in range(counter)]
scenario_variable_df.drop(columns=drop_columns_list, inplace=True)

scenario_variable_df


# Read Transform variables 

In [ ]:
# Get the last upload excel file
real_folder_path= os.path.join(folder_path , folder_name_to_convert)

last_file = get_last_added_file(real_folder_path)


if last_file:
    print("Last added file:", last_file)
else:
    print("No files found in the folder.")

# Get the correct naming for the converted file 
filename_with_extension = os.path.basename(last_file)
splited_filename_with_extension = os.path.splitext(filename_with_extension)
print(splited_filename_with_extension)
filename = splited_filename_with_extension[0] + 'converted' + splited_filename_with_extension[1]

# Possibility to create a thread to run all the code during the wait for the user answer. 
# scenario = input("Give the name of the scenario")
scenario = splited_filename_with_extension[0]

# Read the selected file 
if splited_filename_with_extension[1] == '.xls' or splited_filename_with_extension[1] == '.xlsx':
    scenario_variable_2_df = pd.read_excel(last_file)
elif  splited_filename_with_extension[1] == '.csv': 
    scenario_variable_2_df = pd.read_csv(last_file)


scenario_variable_2_df

In [ ]:
#Rename the column of the file 
columns = scenario_variable_2_df.columns.str.replace('Unnamed:', 'Subscript')
columns = columns.str.replace('.', ' ')

# Define the string you want in column names
search_string = 'Subscript'
counter=0 

for k in range(len(columns)): 
    if search_string in columns[k]: 
        columns.values[k]= search_string + ' ' + str(counter)
        counter+=1



# Create a dictionary using zip() and dictionary comprehension
my_dict = {k: v for k, v in zip(scenario_variable_2_df.columns[2:6], columns[2:6])}
scenario_variable_2_df.rename(columns=my_dict, inplace=True)
scenario_variable_2_df.rename(columns={"Time": "Variable", "Year": "Unit", "Subscript":"Subscript 0"}, inplace=True)



# Insert the three columns in the right place for IAMC format
scenario_variable_2_df.insert(0, 'Model', "WILIAM")
scenario_variable_2_df.insert(1, 'Scenario', scenario)
scenario_variable_2_df.insert(2,"Region","World")


In [ ]:
# Check for duplicate rows
duplicate_rows = scenario_variable_2_df[scenario_variable_2_df.duplicated()]

# Display duplicate rows
print("Duplicate Rows:")
print(duplicate_rows)

# Count the number of duplicate rows
num_duplicate_rows = duplicate_rows.shape[0]
print("Number of Duplicate Rows:", num_duplicate_rows)

scenario_variable_2_df = scenario_variable_2_df.drop_duplicates()
print(len(scenario_variable_2_df))

# Read the CSV file into a pandas DataFrame
data_name_df = pd.read_excel("../Variable_Reference/Variable_name_IAMC.xlsx")
# Remplacer les tirets du bas par des espaces dans la colonne
# data_name_df['WILIAM_variable'] = data_name_df['WILIAM_variable'].str.replace('_', ' ')
# Drop rows with NaN values which corresponds to values not conserv for the final upload of data in IAMC format
data_name_df.dropna(subset=["IAMC_variable"], inplace=True)
data_name_df

In [ ]:
scenario_variable_new_df = pd.DataFrame()
# Create a dict with Wiliam's name as key, IAMC's name as value
IAMC_WILIAM_name_dict = data_name_df.set_index('WILIAM_variable')['IAMC_variable'].to_dict()
IAMC_WILIAM_name_dict['economy_dashboard'] = ''
IAMC_WILIAM_name_dict['economy_dashboard_1R'] = ''
IAMC_WILIAM_name_dict['economy_dashboard_9R'] = ''
IAMC_WILIAM_name_dict['economy_dashboard_EU27'] = ''
print(IAMC_WILIAM_name_dict)
# Replace the variable name used in William to the ones used for IAMC format. 
scenario_variable_2_df['Old_Variable'] = scenario_variable_2_df['Variable']
scenario_variable_2_df['Variable']= scenario_variable_2_df['Variable'].replace(IAMC_WILIAM_name_dict)

# Keep only the rows where the value in column 'Variable' belongs to the list of wanted values 
variable_required_list = data_name_df['IAMC_variable'].to_list()
# scenario_variable_df = scenario_variable_df[scenario_variable_df['Variable'].isin(variable_required_list)]


In [ ]:
scenario_variable_2_df['Variable'].unique()

In [ ]:
variables_used_list = scenario_variable_2_df['Variable'].to_list()
variables_required_not_in = list(set(variable_required_list) - set(variables_used_list))
print('The missing variable in the export dataset are the following:', variables_required_not_in)
print('The number of missing varibles is ', len(variables_required_not_in))

In [ ]:
# Open the text file containing the energy dictionary
with open('../Create_Variable_Dict/energy_dict.txt', 'r') as f:
    # Read the contents of the file
    energy_dict_str = f.read()

# Convert the string representation of the dictionary back to a dictionary object
energy_dict = ast.literal_eval(energy_dict_str)


# Open the text file containing the rest dictionary
with open('../Create_Variable_Dict/rest_dict.txt', 'r') as f:
    # Read the contents of the file
    rest_dict_str = f.read()

# Convert the string representation of the dictionary back to a dictionary object
rest_dict = ast.literal_eval(rest_dict_str)


# Open the text file containing the sectors dictionary
with open('../Create_Variable_Dict/sectors_dict.txt', 'r') as f:
    # Read the contents of the file
    sectors_str = f.read()

# Convert the string representation of the dictionary back to a dictionary object
sectors_dict = ast.literal_eval(sectors_str)

# Open the text file containing the dictionary
with open('../Create_Variable_Dict/country_dict.txt', 'r') as f:
    # Read the contents of the file
    dict_str = f.read()

# Convert the string representation of the dictionary back to a dictionary object
country_Wiliam_dict = ast.literal_eval(dict_str)

# Open the text file containing the dictionary
with open('../Create_Variable_Dict/economy_dashboard_dict.txt', 'r') as f:
    # Read the contents of the file
    economy_dashboard_str = f.read()

# Convert the string representation of the dictionary back to a dictionary object
economy_dashboard_dict = ast.literal_eval(economy_dashboard_str)

# Open the text file containing the dictionary
with open('../Create_Variable_Dict/COICOP_dict.txt', 'r') as f:
    # Read the contents of the file
    COICOP_dict_str = f.read()

# Convert the string representation of the dictionary back to a dictionary object
COICOP_dict = ast.literal_eval(COICOP_dict_str)

# Open the text file containing the dictionary
with open('../Create_Variable_Dict/final_demand_dict.txt', 'r') as f:
    # Read the contents of the file
    final_demand_dict_str = f.read()

# Convert the string representation of the dictionary back to a dictionary object
final_demand_dict = ast.literal_eval(final_demand_dict_str)

# Open the text file containing the dictionary
with open('../Create_Variable_Dict/land_dict.txt', 'r') as f:
    # Read the contents of the file
    land_dict_str = f.read()

# Convert the string representation of the dictionary back to a dictionary object
land_dict = ast.literal_eval(land_dict_str)

# Open the text file containing the dictionary
with open('../Create_Variable_Dict/transport_mode_dict.txt', 'r') as f:
    # Read the contents of the file
    transport_mode_dict_str = f.read()

# Convert the string representation of the dictionary back to a dictionary object
transport_mode_dict = ast.literal_eval(transport_mode_dict_str)


# Open the text file containing the dictionary
with open('../Create_Variable_Dict/transport_power_train_dict.txt', 'r') as f:
    # Read the contents of the file
    transport_power_train_dict_str = f.read()

# Convert the string representation of the dictionary back to a dictionary object
transport_power_train_dict = ast.literal_eval(transport_power_train_dict_str)


# Open the text file containing the dictionary
with open('../Create_Variable_Dict/households_categories_dict.txt', 'r') as f:
    # Read the contents of the file
    households_categories_dict_str = f.read()

# Convert the string representation of the dictionary back to a dictionary object
households_categories_dict = ast.literal_eval(households_categories_dict_str)

# Replace the region name by the ones adapted for Wiliam. 
scenario_variable_2_df['Region'] = scenario_variable_2_df['Region'].replace(country_Wiliam_dict)


In [ ]:
list_of_string = ['Final Energy|Net[per capita]','Primary Energy[per capita]','Primary Energy|Renewable[Share]','Final Energy|Renewable[Share]','Final Energy|Renewable[Share]', 'Emissions|CO2eq[per capita]','Households|Consumer Price[index]','Final Energy[intensity]','Primary Energy Intensity|GDP[Annual change]']
# Fonction pour vérifier la présence d'un mot entier
def contains_word(text, words):
    for elem in words: 
        if elem in text: 
            return True
    return False

# Filtrer le DataFrame en utilisant `apply` avec la fonction lambda
filtered_df = scenario_variable_2_df[scenario_variable_2_df['Variable'].apply(lambda text: contains_word(text, list_of_string))]

print(filtered_df)

In [ ]:
# Dict used to check if the subscripts are a country
country_dict = {
    "EU27": 1,
    "UK": 1,
    "CHINA": 1,
    "EASOC": 1,
    "INDIA": 1,
    "LATAM": 1,
    "RUSSIA": 1,
    "USMCA": 1,
    "LROW": 1,
    "AUSTRIA": 1,
    "BELGIUM": 1,
    "BULGARIA": 1,
    "CROATIA": 1,
    "CYPRUS": 1,
    "CZECH_REPUBLIC": 1,
    "DENMARK": 1,
    "ESTONIA": 1,
    "FINLAND": 1,
    "FRANCE": 1,
    "GERMANY": 1,
    "GREECE": 1,
    "HUNGARY": 1,
    "IRELAND": 1,
    "ITALY": 1,
    "LATVIA": 1,
    "LITHUANIA": 1,
    "LUXEMBOURG": 1,
    "MALTA": 1,
    "NETHERLANDS": 1,
    "POLAND": 1,
    "PORTUGAL": 1,
    "ROMANIA": 1,
    "SLOVAKIA": 1,
    "SLOVENIA": 1,
    "SPAIN": 1,
    "SWEDEN": 1,
}




# Aggregate the subscripts at the end of the variable
def aggregate_variable_name(row, counter):
    for k in range(counter):
        subscript = row["Subscript " + str(k)]

        if k == 0:
            if country_dict.get(subscript) == 1:
                # Give the name of the region for that variable
                row["Region"] = subscript

                row["Subscript 0"] = np.nan
                continue
    
        if not pd.isnull(subscript):
            if energy_dict.get(subscript, None):
                subscript = energy_dict[subscript]
            elif rest_dict.get(subscript, None):
                subscript = rest_dict[subscript]
            elif sectors_dict.get(subscript, None):
                subscript = sectors_dict[subscript]
            elif economy_dashboard_dict.get(subscript, None):
                subscript = economy_dashboard_dict[subscript]
            elif COICOP_dict.get(subscript, None):
                subscript = COICOP_dict[subscript]
            elif final_demand_dict.get(subscript, None):
                subscript = final_demand_dict[subscript]
            elif land_dict.get(subscript, None):
                subscript = land_dict[subscript]
            elif transport_mode_dict.get(subscript, None):
                subscript = transport_mode_dict[subscript]
            elif transport_power_train_dict.get(subscript, None):
                subscript = transport_power_train_dict[subscript]
            elif households_categories_dict.get(subscript, None):
                subscript = households_categories_dict[subscript]
            
            row["Variable"] = row["Variable"] + "|" + subscript
            

    return row


# Apply the previous function on all the lines
scenario_variable_2_df = scenario_variable_2_df.apply(
    aggregate_variable_name, args=(counter,), axis=1
)

# Fonction pour vérifier la présence d'un mot entier
def contains_word(text, words):
    for elem in words: 
        if elem in text: 
            return True
    return False

# Filtrer le DataFrame en utilisant `apply` avec la fonction lambda
filtered_df = scenario_variable_2_df[scenario_variable_2_df['Variable'].apply(lambda text: contains_word(text, list_of_string))]

print(filtered_df)




In [ ]:
# Fonction pour vérifier la présence d'un mot entier
def contains_word(text, words):
    for elem in words: 
        if elem in text: 
            return True
    return False

# Filtrer le DataFrame en utilisant `apply` avec la fonction lambda
filtered_df = scenario_variable_2_df[scenario_variable_2_df['Variable'].apply(lambda text: contains_word(text, list_of_string))]

print(filtered_df)


In [ ]:
"""scenario_variable_new_df= pd.DataFrame()
list_of_string = ['Value|GDP Real']
# Fonction pour vérifier la présence d'un mot entier
def contains_word(text, words):
    for elem in words: 
        if elem in text: 
            return True
    return False

scenario_variable_new_df = scenario_variable_2_df[scenario_variable_2_df['Variable'].apply(lambda text: contains_word(text, list_of_string))]"""


In [ ]:

# List with the words in uppercase which will not be capitalized by the next functions
upper_word_list = ['CO2', 'CH4', 'N2O', 'PFC', 'SF6', 'HFC134a', 'HFC23', 'HFC32', 'HFC125', 'HFC143a', 'HFC152a', 'HFC227ea', 'HFC245ca', 'HFC4310mee', 'CO2eq[per capita]', 'HFC', 'w/o CCS' ,'w/ CCS', 'PV', 'CSP', 'AFOLU', 'AFOFI' , 'CO2eq', 'EROI', 'PV', 'CSP', 'LMO', 'NMC622', 'NMC811', 'NCA', 'LFP', 'LDV','MDV','HDV','NMT','GDP', 'PPP']
vehicule_list = ['gasoline', 'gas' , 'diesel']
conjunctions = ["and", "or", "nor", "but", "so", "for", "yet", "of", "w/", "w/o"]

# Change the format of the string to respect IAMC's format
def transform_string(s):
    # Split the string into words
    if pd.isnull(s): 
        return s
    words = s.split('|')

    capitalized_words = []
    # Capitalize the first letter of each word
    for word in words: 
        
        if any(element in word for element in upper_word_list):
            new_word_list = word.split('_')
            
            word_list=[]
            for new_word in new_word_list: 
                single_element_list =new_word.split(' ')
                
                single_element_list = [single_element.capitalize() if single_element not in conjunctions and single_element not in upper_word_list else single_element for single_element in single_element_list ]
                
                word_list += [' '.join(single_element_list)]
            
            capitalized_words += [' '.join(word_list) + '|']
        else:          
            capitalized_words += [word + '|']
    
    # Join the words with spaces
    transformed_string = ''.join(capitalized_words)
    
    result = ''
    capitalize_next=False
    # Capitalize the letter after each space
    for number,char in enumerate(transformed_string[:-1]):
        if number == 0 and char == '|': 
            continue
        if char == '-':
            capitalize_next = True
            result += char
        elif capitalize_next:
            result += char.upper()
            capitalize_next = False
        else:
            result += char
                
    
    return result


# scenario_variable_new_df = scenario_variable_2_df[scenario_variable_2_df['Variable'].apply(lambda text: contains_word(text, list_of_string))]
# Capitalize each variable's name
scenario_variable_2_df['Variable'] = scenario_variable_2_df['Variable'].apply(transform_string)


In [ ]:
scenario_variable_new_df

In [ ]:
len(scenario_variable_2_df)

In [ ]:
# Fonction pour vérifier la présence d'un mot entier
def contains_word(text, words):
    for elem in words: 
        if elem in text: 
            return True
    return False

# Filtrer le DataFrame en utilisant `apply` avec la fonction lambda
filtered_2df = scenario_variable_2_df[scenario_variable_2_df['Variable'].apply(lambda text: contains_word(text, list_of_string))]

print(filtered_2df)


In [ ]:
# Lignes présentes dans df1 mais pas dans df2
diff_df1 = filtered_2df[~filtered_2df.isin(filtered_df.to_dict(orient='list')).all(axis=1)]

# Lignes présentes dans df2 mais pas dans df1
diff_df2 = filtered_df[~filtered_df.isin(filtered_2df.to_dict(orient='list')).all(axis=1)]

# Affichage des résultats
print("Lignes présentes dans df1 mais pas dans df2:")
print(diff_df1['Variable'].unique())

print("\nLignes présentes dans df2 mais pas dans df1:")
print(diff_df2['Variable'].unique())

# Make modifications on some variables

In [ ]:
# Select rows with the name 'Emission'
emission_rows = scenario_variable_2_df[scenario_variable_2_df['Variable'].str.contains('Emission')]
emission_name_old = emission_rows['Variable'].to_list()
# List of Greenhouses Gases used in Wiliam 
# We differentiate the treatment of HFC and others GHGs because they are handle differently in IAMC Format. 
GHGs_to_check = ['CH4', 'N2O', 'PFC', 'SF6','CO2']
HFC_list = ['HFC134a', 'HFC23', 'HFC32', 'HFC125', 'HFC143a', 'HFC152a', 'HFC227ea', 'HFC245ca', 'HFC4310mee']


# Iterate over the rows to create the name and the unit of emissions correctly. 
# The new list of variables and unit is then updated to the real dataframe.
variable_list = []
unit_list = []

for row_index, row in emission_rows[['Variable','Unit']].iterrows():
    specific_word = 'Emissions'
    ghg_bool = False
    for string in GHGs_to_check:
        
        if string in row['Variable']:
            
            index_emission = row['Variable'].find('Emissions') + len(specific_word)
            index_gas = row['Variable'].find(string)
            row['Variable'] = row['Variable'][:index_emission] + '|' + string + row['Variable'][index_emission:index_gas-1] + row['Variable'][index_gas+len(string):]            
            ghg = string
            ghg_bool = True
        
            break
    
    for string in HFC_list:
        if string in row['Variable']:
            
            index_emission = row['Variable'].find('Emissions') + len(specific_word)
            index_gas = row['Variable'].find(string)
            row['Variable'] = row['Variable'][:index_emission] + '|HFC|' + string + row['Variable'][index_emission:index_gas-1] + row['Variable'][index_gas+len(string):]
            
            break
    index_year = row['Unit'].find('/yr')
    if row['Unit'] == 'Mt/yr' or row['Unit'] == 'Gt/yr':
        if ghg_bool: 
            row['Unit'] = row['Unit'][:index_year] + ' ' + ghg + row['Unit'][index_year:]
        else : 
            row['Unit'] = row['Unit'][:index_year] + ' ' + string + row['Unit'][index_year:]
    variable_list.append(row['Variable'])
    unit_list.append(row['Unit'])

indexes = emission_rows.index

# Assign new values to specific rows in the column for the Variable and Unit 
scenario_variable_2_df.loc[indexes, 'Variable'] = variable_list
scenario_variable_2_df.loc[indexes, 'Unit'] = unit_list

In [ ]:
scenario_variable_2_df['Variable'].to_list()

In [ ]:
# Change Primary Energy Naming
# Select the rows where the primary energy word 
energy_rows = scenario_variable_2_df[scenario_variable_2_df['Variable'].str.contains('Per Capita')]


energy_name_old = energy_rows['Variable'].to_list()

# Function to move the matched string after the specific word
def move_crochets(s):
    specific_word_1 = '[per capita]'
    specific_word_2 = '[intensity]'
    specific_word_3 = '[Share]'
    specific_word_4 = '[index]'
    specific_word_5 = '[per Capita]'
    
    if specific_word_1 in s:
        index_emission = s.find(specific_word_1) 
        
        s = s[:index_emission-1] + s[index_emission+len(specific_word_1):]  + specific_word_1
    
    if specific_word_2 in s:
        index_emission = s.find(specific_word_2) 
        
        s = s[:index_emission-1] + s[index_emission+len(specific_word_2):]  + specific_word_2
    
    if specific_word_3 in s:
        index_emission = s.find(specific_word_3) 
        
        s = s[:index_emission-1] + s[index_emission+len(specific_word_3):]  + '[Share]'

    if specific_word_4 in s:
        index_emission = s.find(specific_word_4) 
        
        s = s[:index_emission-1] + s[index_emission+len(specific_word_4):]  + '[index]'
    
    if specific_word_5 in s:
        index_emission = s.find(specific_word_5) 
        
        s = s[:index_emission-1] + s[index_emission+len(specific_word_5):]  + specific_word_5

    return s

# Apply the function to the 'Variable' column
"""energy_rows['Variable'] = energy_rows['Variable'].apply(move_crochets)
energy_name_new = energy_rows['Variable'].to_list()

energy_per_capita_dict = dict(zip(energy_name_old, energy_name_new))

scenario_variable_2_df['Variable'] = scenario_variable_2_df['Variable'].replace(energy_per_capita_dict)"""
scenario_variable_2_df['Variable'] = scenario_variable_2_df['Variable'].apply(move_crochets)


In [ ]:
# Change Secondary Energy Naming 
secondary_energy_rows = scenario_variable_2_df[scenario_variable_2_df['Variable'].str.contains('Secondary Energy')]


# Change Secondary Energy Naming 
# Select the rows for Primary Energy Price 
price_energy_rows = scenario_variable_2_df[scenario_variable_2_df['Variable'].str.contains('Price')]
price_energy_rows = price_energy_rows[price_energy_rows['Variable'].str.contains('Primary Energy')]
price_energy_rows

def remove_characters_between_indices(input_string, start_index, end_index):
    """
    Remove characters between two given indices of a string.

    Args:
    - input_string (str): The original string.
    - start_index (int): The index of the first character to remove (inclusive).
    - end_index (int): The index of the last character to remove (inclusive).

    Returns:
    - str: The modified string with characters removed between the specified indices.
    """
    if start_index >= end_index:
        return input_string  # Return the original string if start index is greater than or equal to end index
    
    # Ensure indices are within the range of the string
    start_index = max(start_index, 0)
    end_index = min(end_index, len(input_string) - 1)
    
    # Construct the new string by concatenating substrings before and after the characters to be removed
    return input_string[:start_index] + input_string[end_index + 1:]

def remove_string_between_pipe(input_string):
    """
    Remove the string between the second and third '|' characters in the input string.

    Args:
    - input_string (str): The original string.

    Returns:
    - str: The modified string with the content between '|' characters removed.
    """
    second_occurrence = input_string.find('|', input_string.find('|') + 1)
    third_occurrence = input_string.find('|', second_occurrence + 1)-1
    
    # Remove characters between the '|' characters using the previous function
    result = remove_characters_between_indices(input_string, second_occurrence, third_occurrence)

    # Remove CCS extension because it is not relevant 
    result = result.replace('|w/ CCS', '').replace('|w/o CCS', '')
    
    return result

"""price_energy_rows['Variable'] = price_energy_rows['Variable'].apply(remove_string_between_pipe)
indexes = price_energy_rows.index
# Assign new values to specific rows in the column for the Variable and Unit 
scenario_variable_2_df.loc[indexes, 'Variable'] = price_energy_rows['Variable']
"""


In [ ]:
import csv
# Remove the duplication on the variable column 
scenario_variable_df = scenario_variable_df.drop_duplicates(subset=["Variable"])
print(len(scenario_variable_df))

# Step 1: Get the column of old name
data_first_column = scenario_variable_df["Variable"].to_list()


# Remove the duplication of the Variable column except when the region is 'World'
# Assign a temporary index column to keep track of the original order
scenario_variable_2_df['temp_index'] = scenario_variable_2_df.index


# Drop duplicates based on 'Variable' for non-'World' rows
non_world_rows = scenario_variable_2_df.drop_duplicates(subset=['Variable', 'Old_Variable', 'Subscript 1', 'Subscript 2', 'Subscript 3'])



# Step 2: Get the column of old name
data_second_column = non_world_rows["Variable"].to_list()
print(len(data_second_column))



# Write the new data along with the existing data
with open("example.csv", "w", newline="") as csvfile:
    csvwriter = csv.writer(csvfile)
    for k in range(len(data_second_column)):
        csvwriter.writerow([data_first_column[k], data_second_column[k]])
    for k in range(len(data_second_column), len(data_first_column)): 
        csvwriter.writerow([data_first_column[k]])
